# Ex2

<table><tr>
    <td><img src="image/esi-sba.png" width="100" height="100"></td>
    <td><img src="image/mongo.png" width="100" height="100"></td>
    </tr></table>

### Lancer un conteneur de MONGO
   
! docker-compose up –d


### Importation d’un fichier JSON:
* décompresser les fichier dblp.zip
*  docker cp dblp.json mongo-esi:/dblp.json
* docker exec mongo-esi  mongoimport -d dblp -c publis  /dblp.json
 

#### installer pymongo afin de se connecter au cluster mongodb

! pip install pymongo

In [130]:
from pymongo import MongoClient

### Créer un Mongo Client afin d'exécuter nos requêtes sur MONGODB Server

In [131]:
client = MongoClient('mongodb://127.0.0.1:27017/')

###  Afficher les différentes  databases

In [132]:
client.list_database_names()

['admin', 'config', 'db1', 'dblp', 'local']

### Créer une "database variable" pour travailler avec la BDD "dblp"

In [133]:
db = client["dblp"]

### Récupérer les collections de la BDD "dblp"

In [134]:
db.list_collection_names()

['publis']

###  afficher le #Docs dans la collection "publis"

In [135]:
db["publis"].count_documents({})

118015

In [136]:
db["publis"].find_one()

{'_id': 'series/cogtech/Zancanaro12',
 'type': 'Article',
 'title': 'Shared Interfaces for Co-located Interaction.',
 'pages': {'start': 71, 'end': 88},
 'year': 2012,
 'booktitle': 'Ubiquitous Display Environments',
 'url': 'db/series/cogtech/364227662.html#Zancanaro12',
 'authors': ['Massimo Zancanaro']}

##  Visualiser les données avec compass
### Installer compass, un logiciel pour manipuler MongoDB
https://www.mongodb.com/try/download/compass

### Donner le titre des publications dont le type est « Book »

In [148]:
result = list(db.publis.find(
    { "type": "Book" },
    { "_id": 0, "title": 1 }
))
print(result[:5])

[{'title': 'Foundations of Rule Learning'}, {'title': 'Ubiquitous Display Environments'}, {'title': 'Language Processing with Perl and Prolog - Theories, Implementation, and Application'}, {'title': 'SemProM - Foundations of Semantic Product Memories for the Internet of Things'}, {'title': 'SmartKom: Foundations of Multimodal Dialogue Systems'}]


### Donner le titre des publications ayant «publisher » et « editor » renseignés



In [149]:
result = list(db.publis.find(
    { "publisher": { "$exists": True }, "editor": { "$exists": True } },
    { "_id": 0, "title": 1 }
))
print(result[:5])


[{'title': 'Ubiquitous Display Environments'}, {'title': 'SemProM - Foundations of Semantic Product Memories for the Internet of Things'}, {'title': 'SmartKom: Foundations of Multimodal Dialogue Systems'}, {'title': 'Artificial General Intelligence'}, {'title': 'Machine Learning Techniques for Multimedia - Case Studies on Organization and Retrieval'}]


### 	Donner le titre des publications qui sont publiées entre 2005 et 2007.

In [150]:
result = list(db.publis.find(
    { "year": { "$gte": 2005, "$lte": 2007 } },
    { "_id": 0, "title": 1, "year": 1 }
))
print(result[:5])


[{'title': 'Speech Recognition.', 'year': 2006}, {'title': 'Dialogue Systems Go Multimodal: The SmartKom Experience.', 'year': 2006}, {'title': 'SmartKom-Public.', 'year': 2006}, {'title': '3D Simulation: the Key to A.I.', 'year': 2007}, {'title': 'Discourse Modeling.', 'year': 2006}]


### Donner le nombre des publications dont le titre contient « Ontology » 


In [152]:
count = db.publis.count_documents(
    { "title": { "$regex": "Ontology", "$options": "i" } }
)
print(f"The number of publications with 'Ontology' in the title is {count}.")


The number of publications with 'Ontology' in the title is 731.


### Donner le titre des publications publiées par l’auteur «Mimoun Malki »



In [153]:
result = list(db.publis.find(
    { "authors": "Mimoun Malki" },
    { "_id": 0, "title": 1 }
))
print(result[:5])


[{'title': 'Existential Dependency Driven Approach for Extracting Views from Domain Ontology.'}, {'title': 'Automated Migration of Data-Intensive Web Pages into Ontology-Based Semantic Web: A Reverse Engineering Approach.'}, {'title': 'WordNet based Cross-Language Text Categorization.'}, {'title': 'Adapting WordNet to the Medical Domain using Lexicosyntactic Patterns in the Ohsumed Corpus.'}, {'title': 'Towards a multi-representation ontology-based information systems mediation.'}]


### Donner le titre des publications dont  «Mimoun Malki » est premier auteur

In [154]:
result = list(db.publis.find(
    { "authors.0": "Mimoun Malki" },
    { "_id": 0, "title": 1 }
))
print(result[:5])

[{'title': 'Static and Dynamic Reverse Engineering of Relational Database Applications: A Form-Driven Methodology.'}]


### Liste distincte des auteurs



In [143]:
authors = db.publis.distinct("authors")
num_authors = len(authors)
print(f"There are {num_authors} authors in the collection.")
for i in range(5):
    print(authors[i])


There are 158758 authors in the collection.
??igo Garc?
?agatan Taskin
?agatay Dikici
?aglayan Erdem
?agri ?zgenc Etemoglu


### Retourner le titre et l’année des publications  de type article, et trier sur l'année dans le l’ordre décroissant  


In [155]:
result = list(db.publis.find(
    { "type": "Article" },
    { "_id": 0, "title": 1, "year": 1 }
).sort("year", -1))
print(result[:5])


[{'title': 'Reliable Implementation of Robust Adaptive Topology Control.', 'year': 2014}, {'title': 'Towards ICT Revolution in Healthcare: Present and Perspectives for Electronic Healthcare Record Systems.', 'year': 2014}, {'title': 'Government and Social Media: A Case Study of 31 Informational World Cities.', 'year': 2014}, {'title': 'A new importance-sampling ML estimator of time delays and angles of arrival in multipath environments.', 'year': 2014}, {'title': 'Sharp performance bounds for graph clustering via convex optimization.', 'year': 2014}]


### Donner le nombre publications par type



In [156]:
result = list(db.publis.aggregate([
    { "$group": { "_id": "$type", "count": { "$sum": 1 } } }
]))
print(result)


[{'_id': 'Book', 'count': 11074}, {'_id': 'Phd', 'count': 6950}, {'_id': 'Article', 'count': 99991}]


### Pour chaque auteur le nombre des publications. Trier par ordre décroissant




In [157]:
result = list(db.publis.aggregate([
    { "$unwind": "$authors" },
    { "$group": { "_id": "$authors", "count": { "$sum": 1 } } },
    { "$sort": { "count": -1 } }
]))
print(result[:5])


[{'_id': 'Georgios B. Giannakis', 'count': 95}, {'_id': 'Alberto L. Sangiovanni-Vincentelli', 'count': 91}, {'_id': 'Oscar Castillo', 'count': 87}, {'_id': 'Gert-Jan de Vreede', 'count': 82}, {'_id': 'Hermann Ney', 'count': 82}]


## le nombre de pages moyen par type

In [158]:
result = list(db.publis.aggregate([
    { 
        "$addFields": { 
            "numPages": { "$subtract": ["$pages.end", "$pages.start"] } 
        } 
    },
    { 
        "$group": { 
            "_id": "$type", 
            "avgPages": { "$avg": "$numPages" } 
        } 
    }
]))
print(result)

[{'_id': 'Phd', 'avgPages': 195.12232415902142}, {'_id': 'Article', 'avgPages': 3.58381847049484}, {'_id': 'Book', 'avgPages': 267.131973605279}]
